# Junior Doctor Joins the Scottish Deanery


### Imports

In [3]:
from aries_cloudcontroller import AriesAgentController
import os
from termcolor import colored

### Initialise the Agent Controller

In [4]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

Initialising a controller with admin api at http://scottish-deanery-agent:3021 and an api key of DeaneryAPIKey


### Start a Webhook Server

In [5]:
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

await agent_controller.init_webhook_server(webhook_host, webhook_port)

print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


## Register Agent Event Listeners

You can see some examples within the webhook_listeners recipe. Copy any relevant cells across and fill in additional logic as needed.

In [6]:
listeners = []

In [7]:
## YOUR LISTENERS HERE
# Receive connection messages
def connections_handler(payload):
    state = payload['state']
    connection_id = payload["connection_id"]
    their_role = payload["their_role"]
    routing_state = payload["routing_state"]
    
    print("----------------------------------------------------------")
    print("Connection Webhook Event Received")
    print("Connection ID : ", connection_id)
    print("State : ", state)
    print("Routing State : ", routing_state)
    print("Their Role : ", their_role)
    print("----------------------------------------------------------")
    if state == "active":
        # Your business logic
        print(colored("Connection ID: {0} is now active.".format(connection_id), "green", attrs=["bold"]))

connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

listeners.append(connection_listener)

In [8]:
def issuer_handler(payload):
    connection_id = payload['connection_id']
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    print("\n---------------------------------------------------\n")
    print("Handle Issue Credential Webhook")
    print(f"Connection ID : {connection_id}")
    print(f"Credential exchange ID : {exchange_id}")
    print("Agent Protocol Role : ", role)
    print("Protocol State : ", state )
    print("\n---------------------------------------------------\n")
    
    
    if state == "offer_sent":
        proposal = payload["credential_proposal_dict"]
        attributes = proposal['credential_proposal']['attributes']

        print(f"Offering credential with attributes  : {attributes}")
        ## YOUR LOGIC HERE
    elif state == "request_received":
        print("Request for credential received")
        ## YOUR LOGIC HERE
    elif state == "credential_sent":
        print("Credential Sent")
        ## YOUR LOGIC HERE
    
issuer_listener = {
    "topic": "issue_credential",
    "handler": issuer_handler
}

listeners.append(issuer_listener)

In [17]:

def verifier_proof_handler(payload):
    role = payload["role"]
    connection_id = payload["connection_id"]
    pres_ex_id = payload["presentation_exchange_id"]
    state = payload["state"]
    print("\n---------------------------------------------------------------------\n")
    print("Handle present-proof")
    print("Connection ID : ", connection_id)
    print("Presentation Exchange ID : ", pres_ex_id)
    print("Protocol State : ", state)
    print("Agent Role : ", role)
    print("Initiator : ", payload["initiator"])
    print("\n---------------------------------------------------------------------\n")
    

    if state == "request_sent":
        print("Presentation Request\n")
        print(payload["presentation_request"])
        print("\nThe presentation request is encoded in base64 and packaged into a DIDComm Message\n")
        print(payload["presentation_request_dict"])
        print("\nNote the type defines the protocol present-proof and the message request-presentation\n")
        print(payload["presentation_request_dict"]["@type"])
    elif state == "presentation_received":
        print("Presentation Received")
        print("We will not go into detail on this payload as it is comparable to the presentation_sent we looked at in the earlier cell.")
        print("This is the full payload\n")
        print(payload)
    else:
        print("Paload \n")
        print(payload)
        
verifier_listener = {
    "topic": "present_proof",
    "handler": verifier_proof_handler
}

listeners.append(verifier_listener)

In [9]:
agent_controller.register_listeners(listeners)

## Store Issuing Schema and Cred Def Identifiers

If you intend for this agent to issue credentials you should first initialise your agent as an issuer and author the relevant identifiers to the public ledger. The issuer_initialisation recipe notebook can be duplicated and used as a starting point.

Once schema and cred def identifiers are created copy across and store in variables as illustrated in the cell below. Be sure to use unique names for each variable.

In [11]:
rtw_schema_id = 'MKDB7pQgw1KGJZ1UtJHCkY:2:UK Right to Work:0.0.1'
rtw_cred_def_id = 'MKDB7pQgw1KGJZ1UtJHCkY:3:CL:216271:doctor-onboarding'

dbs_schema_id = 'MKDB7pQgw1KGJZ1UtJHCkY:2:DBS Check:0.0.1'
dbs_cred_def_id = 'MKDB7pQgw1KGJZ1UtJHCkY:3:CL:216272:doctor-onboarding'



## Before Joining the Scottish Deanery the Doctor Must Demonstrate they Have a GMC Licence

The deanery challenges them with the following proof request

In [12]:
import time
# The GMC is the only entity able to issue GMC licences in the UK Healthcare system
gmc_schema_id = 'TDAbSf3Uqebg8N4XvybMbg:2:GMC Licence:0.0.1'

gmc_issuer_did = "TDAbSf3Uqebg8N4XvybMbg"

# cred_def_id = "<SOME CRED DEF>"

# Define the list of attributes and restrictions under which each attribute was issued that a prover must satisfy with a presentation
# NOTE: if identifying a schema or credential definition then the attribute name must be contained within the corresponding schema.
req_attrs = [
    {"name": "Headshot", 
     "restrictions": [{"schema_id": gmc_schema_id, "issuer_did": gmc_issuer_did}],
     "non_revoked": {"to": int(time.time() - 1)}
    },
    {"name": "Name", 
     "restrictions": [{"schema_id": gmc_schema_id, "issuer_did": gmc_issuer_did}],
     "non_revoked": {"to": int(time.time() - 1)}
    },
    {"name": "GMC Number", 
     "restrictions": [{"schema_id": gmc_schema_id, "issuer_did": gmc_issuer_did}],
     "non_revoked": {"to": int(time.time() - 1)}
    },
]

# We could extend this to request the name attribute aswell if we wanted.


gmc_licence_proof_request = {
    "name": "Proof of GMC Licence",
    "version": "1.0",
    "requested_attributes": {
        # They must follow this uuid pattern
        # Note that req_attr['name'] gets the attribute name of each object. E.g. domain and name in this case
        f"{req_attr['name']}":
        req_attr for req_attr in req_attrs
    },
    # Predicates allow us to specify range proofs or set membership on attributes. For example greater than 10.
    # We will ignore these for now.
    "requested_predicates": {
#         f"0_{req_pred['name']}_GE_uuid":
#         req_pred for req_pred in req_preds
    },
    # You can also request the entire proof request be non-revoked
#     "non_revoked":  {"to": int(time.time())}
}

## Establish Connection

Before you can issue a credential you must first establish a connection across which the credential will be issued to a holder. (see recipes/connection)

In [13]:
# Alias for invited connection
alias = "Junior Doctor"
auto_accept = "true"
# Use public DID?
public = "false"
# Should this invitation be usable by multiple invitees?
multi_use = "false"

invitation_response = await agent_controller.connections.create_invitation(alias, auto_accept, public, multi_use)
# Is equivalent to above. Arguments are optionally
# invitation_response = await agent_controller.connections.create_invitation()



# You will use this identifier to issue a credential across this connection
connection_id = invitation_response["connection_id"]

----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  466d8ef4-5a5a-46ef-beda-b620130f91ca
State :  invitation
Routing State :  none
Their Role :  invitee
----------------------------------------------------------


## Share Invitation Object with External Agent

In [14]:
invitation = invitation_response["invitation"]
## Copy this output
print(invitation)

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '5c322c3e-dddd-4711-82d4-d51b8adf7732', 'label': 'Scottish Deanery', 'recipientKeys': ['BnPZDJQPvd1ngtBfZ3HDsPmQ24sdeZ7ZemyDu73hTavf'], 'serviceEndpoint': 'https://136f0239772c.ngrok.io'}
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  466d8ef4-5a5a-46ef-beda-b620130f91ca
State :  request
Routing State :  none
Their Role :  invitee
----------------------------------------------------------
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  466d8ef4-5a5a-46ef-beda-b620130f91ca
State :  response
Routing State :  none
Their Role :  invitee
----------------------------------------------------------
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  466d8ef4-5a5a-46ef-beda-b620130f91ca
State :  active
Routing State :  none
Their Rol

## Display Invite as QR Code

This is useful if you wish to issue a credential to a mobile wallet.

In [ ]:
import qrcode
# Link for connection invitation
invitation_url = invitation_response["invitation_url"]
# Creating an instance of qrcode
qr = qrcode.QRCode(
        version=1,
        box_size=5,
        border=5)
qr.add_data(invitation_url)
qr.make(fit=True)
img = qr.make_image(fill='black', back_color='white')
img

## Request Proof of GMC Licence

In [16]:
proof_request = {
    "comment": "Please prove you have a GMC Licence",
    "connection_id": connection_id,
    "proof_request": gmc_licence_proof_request,
    # Do you want your agent to trace this request (for debugging)
    "trace": True
}

proof_request_response = await agent_controller.proofs.send_request(proof_request)

## Get Presentation Exchange Record

This record keeps track of the current state of the presentation protocol, which must be in the `presentation_received` state before the presentation can be verified.

Note: This could also happen in the webhook logic.

In [21]:
response = await agent_controller.proofs.get_records()
record = response["results"][0]
presentation_exchange_id = record["presentation_exchange_id"]

In [ ]:
print(record)

## Verify Presentation



In [22]:
verified_response = await agent_controller.proofs.verify_presentation(presentation_exchange_id)

verified = verified_response["verified"]

ERROR:__name__:Error during POST /present-proof/records/1ed08e32-3c25-45e0-83b0-ab26f71ae6d3/verify-presentation: 500, message='Internal Server Error', url=URL('http://scottish-deanery-agent:3021/present-proof/records/1ed08e32-3c25-45e0-83b0-ab26f71ae6d3/verify-presentation')


ClientResponseError: 500, message='Internal Server Error', url=URL('http://scottish-deanery-agent:3021/present-proof/records/1ed08e32-3c25-45e0-83b0-ab26f71ae6d3/verify-presentation')

## Populate Credential Attributes

Before you can issue a credential, you must define the values that will be issued in this credential. The attribute names **MUST** match those in the schem identified by the <schema_id> value.

Make sure to change all code enclosed with <>.


In [ ]:
<attr_1>=input("Please enter the value for <attr_1>: ")
<attr_2>=input("Please enter the value for <attr_2>: ")
credential_attributes = [
    {"name": "<attr_1>", "value": <attr_1>},
    {"name": "<attr_2>", "value": <attr_1>},
]
print(credential_attributes)

## Send Credential

This is the easiest way to issue a credential because it automates the rest of the protocol steps. 

Note: The `connection_id` must be in the active state before a credential can be sent.

In [ ]:
# Do you want the ACA-Py instance to trace it's processes (for testing/timing analysis)
trace = False
comment = ""
# Remove credential record after issued?
auto_remove = True

# Change <schema_id> and <cred_def_id> to correct pair. Cred_def_id must identify a definition to which your agent has corresponding private issuing key.
send_cred_response = await agent_controller.issuer.send_credential(connection_id, <schema_id>, <cred_def_id>, credential_attributes, comment, auto_remove, trace)

# Note last three args are optional.
# await agent_controller.issuer.send_credential(connection_id, <schema_id>, <cred_def_id, credential_attributes)

## Optional: Revoke Credential

Only possible if the credential issued used a revocable credential definition. (see support_revocation flag in write_cred_def() function).

There are two approachs to revoke an revocable credential

### Option 1: Use credential Exchange ID

Note: In a SSI application as an issuer you are might want to keep track of the identifiers for credential echange records. These can be used to retrieve records stored by the agent and also be used to identify a credential to revoke
Note though that applications might not wish to retain these records as they contain sensitive PII

In [35]:
record_id = send_cred_response["credential_exchange_id"]
try:
    record = await agent_controller.issuer.get_record_by_id(record_id)
except:
    print("Record not found. Did your agent automatically remove it?")

if record:
    # For revoke_credential() you only need to provide (cred_ex_id) OR (rev_reg_id AND cred_rev_id).
    response = await agent_controller.revocations.revoke_credential(record_id, publish=True)
    response

### Option 2: Remove Credential using the Registry and Credential Revocation ID

Does not currently work because of bug in code

In [15]:
registry = await agent_controller.revocations.get_active_revocation_registry_by_cred_def(cred_def_id)
print(registry)

{'result': {'issuer_did': '3JgxmaQ5kPctWA9AeVK4Uy', 'updated_at': '2021-05-10 14:56:03.084788Z', 'state': 'active', 'revoc_reg_id': '3JgxmaQ5kPctWA9AeVK4Uy:4:3JgxmaQ5kPctWA9AeVK4Uy:3:CL:211160:default:CL_ACCUM:ca02d4b5-d9cd-4c01-8782-8b9cd69ba802', 'tag': 'ca02d4b5-d9cd-4c01-8782-8b9cd69ba802', 'created_at': '2021-05-10 14:55:57.158589Z', 'max_cred_num': 1000, 'revoc_reg_entry': {'ver': '1.0', 'value': {'accum': '21 12AF91202A3598FDEE55F77A006495DF1BC7B5854847AAA6E349346182D194904 21 12592B44CE3989F6E65123DAF954705473B6DDC6AD88FDE8A0F996914C92A94EC 6 6B26EF8D9B0F6326AF96E5C8F0EBA0EE998FD16442BE0F5DF2946F72CE82E7FF 4 1C2BCCFC10A5D83CFA4C706865AB3A31400BDAAAB359141064DD2D68BEB55E22 6 8068EEB392199785B0D7059CFDE524BF95AA533B2EDA0FD5363002B6F69ADA0A 4 24AA66458B0519040DEEB0BF17FAD971D3105213DFCAE88157F526D08EB570FD'}}, 'cred_def_id': '3JgxmaQ5kPctWA9AeVK4Uy:3:CL:211160:default', 'revoc_def_type': 'CL_ACCUM', 'revoc_reg_def': {'ver': '1.0', 'id': '3JgxmaQ5kPctWA9AeVK4Uy:4:3JgxmaQ5kPctWA9AeV

In [26]:
revoc_reg_id = registry["result"]["revoc_reg_id"]

In [28]:
# Public revocation registry update to ledger
publish = True
# Believe this is index in rev registry starting from 1
cred_rev_id = 1
response = await agent_controller.revocations.revoke_credential(cred_rev_id = cred_rev_id, rev_reg_id = revoc_reg_id, publish=True)

ClientResponseError: 422, message='Unprocessable Entity', url=URL('http://alice-agent:3021/revocation/revoke')

Error during POST /revocation/revoke: 422, message='Unprocessable Entity', url=URL('http://alice-agent:3021/revocation/revoke')


## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [ ]:
await agent_controller.terminate()